In [16]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append("../")

import torch
from transformers import AutoTokenizer
from jiwer import wer

from src.model_2 import chained_seq2seq
from src.utils import STTDataModule

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [10]:
dm = STTDataModule(device=device)
dm.setup("fit")
dm.setup("test")

train_dataloader = dm.train_dataloader()
val_dataloader = dm.val_dataloader()
test_dataloader = dm.test_dataloader()

In [31]:
tokenizer_encoder = AutoTokenizer.from_pretrained("../model/opus-mt-nl-en")
tokenizer_decoder = AutoTokenizer.from_pretrained("../model/opus-mt-en-nl")
model = chained_seq2seq()
model.to(device)
model.eval()
print()

In [11]:
samp_src, samp_tgt, samp_en = next(iter(test_dataloader))
src_ids = tokenizer_encoder(samp_src, return_tensors="pt", padding=True).input_ids.to(device)
tgt_ids = tokenizer_encoder(samp_tgt, return_tensors="pt",padding=True).input_ids.to(device)

interim_attn = tokenizer_encoder(samp_src, return_tensors="pt", padding=True).attention_mask.to(device)
interim_ids = tokenizer_decoder(samp_en, return_tensors="pt", padding=True).input_ids.to(device) 

In [32]:
out_interim, out_final = model(src_ids, interim_ids, interim_attn, tgt_ids)

2022-10-30 17:26:30.910 | INFO     | src.model_2:forward:27 - source batch size = torch.Size([16, 23])
2022-10-30 17:26:30.911 | INFO     | src.model_2:forward:28 - target batch size = torch.Size([16, 26])
2022-10-30 17:26:30.911 | INFO     | src.model_2:forward:29 - interim batch size = torch.Size([16, 21])
2022-10-30 17:26:30.911 | INFO     | src.model_2:forward:30 - interim mask size = torch.Size([16, 23])
2022-10-30 17:26:30.938 | INFO     | src.model_2:forward:36 - middle_batch = ['the ab are▁inhabitants are the▁original▁inhabitants of astralia', 'my▁keyboard is are', '▁She had▁damaged the bank with her skateboard', '▁This▁summit▁has not met▁expectations in▁every▁respects', '▁where▁yougonna▁get▁your▁maintenance▁done', '▁Commissioner▁you for▁your▁very▁precise▁answers', '▁After▁reading▁many▁reviews had▁finally▁dropped her▁eye on a▁laptop with a▁keyboard▁key▁keyboard', 'I▁make▁urge to the to▁makeize the of', '▁What▁does▁this▁question▁mean▁what▁what▁what▁what▁what▁what▁what', 'theruno

In [26]:
loss_interim, logits_interim = out_interim[:2]
loss_final, logits_final = out_final[:2]
print(loss_interim, loss_final)
generated = model.generate(logits_final)


tensor(5.7072, device='cuda:0', grad_fn=<NllLossBackward0>) tensor(6.3065, device='cuda:0', grad_fn=<NllLossBackward0>)


In [27]:
wer(samp_tgt, generated)

1.3776223776223777

In [28]:
generated

['De abbbtainzen zijn de oorspronkelijke bewoners van Astraalië de de de de de de',
 'Mijn toetsenbord is mijn mijn mijn mijn mijn mijn mijn mijn mijn mijn mijn mijn mijn mijn',
 'Ze had de bank beschadigd met haar skateboard Ze Ze Ze Ze Ze',
 'Deze top heeft niet in alle opzichten aan de verwachtingen voldaan De De De De De De De De De De',
 'waar je jeij je onderhoud doen waar waar waar waar waar waar waar waar waar waar waar waar waar',
 'de commissaris,dankt voor uw zeer nauwkeurige antwoorden De De De De De De De De De De Mijnheer',
 'Na het lezen van veel rec had eindelijk eindelijk haar oog laten vallen op een laptop met een toetsenyy toetsenbord',
 'Ik doe de de de tegen dat de de van te makengradstr dat De De De De',
 'Wat betekent deze in in wat wat wat wat wat wat wat Wat Wat Wat Wat Wat Wat Wat Wat Wat Wat Wat Wat Wat Wat',
 'derunroepons op de de de de de de de de de de de de de de de',
 'Wij wij de te te verminderen of proberen wij energie energieverbruik terug te dringen

In [29]:
samp_tgt

['de aboriginals zijn de oorspronkelijke bewoners van australië',
 'mijn toetsenbord zit vol stof',
 'ze had de bank beschadigd met haar skateboard',
 'deze top heeft niet in alle opzichten aan de verwachtingen voldaan',
 'waar laat jij je onderhoud doen',
 'mijnheer de commissaris bedankt voor uw bijzonder nauwkeurige antwoorden',
 'na het lezen van vele beoordelingen had ze eindelijk haar oog laten vallen op een laptop met een qwerty toetsenbord',
 'ik maak er ernstig bezwaar tegen wanneer men een volk tracht te demoniseren',
 'wat houdt dit vraagstuk in',
 'de tampons zijn op',
 'proberen wij kooldioxideemissies te beperken of proberen wij het energiegebruik terug te dringen',
 'marijke kent olivier nu al meer dan twee jaar',
 'mijn partij vindt dat onverkwikkelijk',
 'het voeren van brood aan eenden is eigenlijk ongezond voor de beesten',
 'juni dit jaar is veel te laat',
 'parket moet je stofzuigen tegels moet je dweilen']

In [37]:
torch.cuda.empty_cache()